## Import packages

In [ ]:
# Load custom libraries
import os
import sys
module_path = os.path.abspath('../')
sys.path.append(module_path)
from mobile_capacity.capacity import Capacity
from mobile_capacity.datastorage import DataStorage
from mobile_capacity.entities.pointofinterest import PointOfInterestCollection
from mobile_capacity.entities.cellsite import CellSiteCollection
from mobile_capacity.spatial import plot_layers

## Load input data

In [ ]:
# Get the current directory
current_dir = os.getcwd()
root_dir = os.path.dirname(current_dir)
data_dir = os.path.join(root_dir, 'data')
logs_dir = os.path.join(root_dir, 'logs')
print(f"Check that this is root directory:{root_dir}")
print(f"Check that this is data directory:{data_dir}")
print(f"Check that this is logs directory:{logs_dir}")

In [ ]:
# Provide country code (ISO-3)
# ISO-3 country codes can be found here https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3
country_code = ''

In [ ]:
# Plug in names of datasets found within the input folder data/input_data/<country_code>/
dataset_ids = dict(
    pointofinterest = 'points-of-interest.csv',
    cellsite = 'cell-sites.csv')

In [ ]:
# Load data into the data collections
data_storage = DataStorage(data_dir=data_dir)

data_collections = dict(
    pointofinterest = PointOfInterestCollection(),
    cellsite = CellSiteCollection(),
)

for data_category in dataset_ids.keys():
    filepath = f"input_data/{country_code}/{dataset_ids[data_category]}"
    entity_records = data_storage.load_data(filepath)
    data_collections[data_category].load_from_records(entity_records)

In [ ]:
# Inspect the data collections
data_collections

## Set analysis parameters

In [ ]:
params = {
        ### Network Configuration ###
        'bw_L850': 5, # MHz on L700 to L900 spectrum bandwidth
        'bw_L1800': 10, # MHz on L1800 spectrum bandwidth
        'bw_L2600': 20, # MHz on L2600 spectrum bandwidth
        'rb_num_multiplier': 5, # Resource block multiplier
        'cco': 18, # Control channel overheads in %
        'sectors_per_site': 3, # Number of frequency bands on site
        'angles_num': 360, # Set the number of angles to be used for azimuth analysis  
        'rotation_angle': 60, # Define the rotation angle to create a sector +/-rotation_angle degrees clockwise and counter-clockwise  

        ### POI configuration ###
        'dlthtarg': 20, # Download throughput target in Mbps.

        ### Population information ###
        'mbb_subscr': 113, # Active mobile-broadband subscriptions per 100 people, source: https://datahub.itu.int/data/?e=701&c=&i=11632&u=per+100+people
        'oppopshare': 50, # % of Population on Operator
        'dataset_year': 2020, # Year of the population dataset
        'one_km_res': True, # Use 1km resolution population data
        'un_adjusted': True, # Use adjusted population data

        ### Mobile coverage radius ###
        'min_radius': 1000, # meters, minimum radius around cell site location for population calculation
        'max_radius': 2000,  # meters, maximum radius should be divisible by 1000; maximum radius around cell site location for population calculation
        'radius_step': 500, # meters, radius step size for population calculation

        ### Avg user traffic profile ###
        'nbhours': 10, # number of non-busy hours per day
        'nonbhu': 50, # Connection usage in non-busy hour in % 
    }

## Create an instance of the Capacity class

In [ ]:
# Create an instance of the Capacity class
mobilecapacity = Capacity(country_code=country_code,
                          poi=data_collections['pointofinterest'],
                          cellsites=data_collections['cellsite'],
                          data_dir=data_dir,
                          logs_dir=logs_dir,
                          enable_logging=True,
                          use_secure_files=True,
                          **params)

In [ ]:
print(f"The mobile broadband internet traffic per subscription per month in {mobilecapacity.country_code} is: {round(mobilecapacity.udatavmonth_pu,0)} GB per month according to {mobilecapacity.udatavmonth_year} data.")

## Run buffer analysis for all POIs

In [ ]:
# Run buffer analysis
buffer_areas, poi_sufcapch_result = mobilecapacity.calculate_buffer_areas()

In [ ]:
# Buffer areas output data sample
buffer_areas["rings"].head(2)

In [ ]:
# POI capacity sufficiency output data sample
poi_sufcapch_result.head(2)

In [ ]:
# Number of True and False POI capacity sufficiency checks
poi_sufcapch_result.sufcapch.value_counts()

## Charts

In [ ]:
plot_layers(mobilecapacity, poi_sufcapch_result, buffer_areas, plot="buffers")

## Save results

In [ ]:
file_path = os.path.join(data_dir, 'output_data', country_code, 'poi-capacity-sufficiency.csv')
os.makedirs(os.path.dirname(file_path), exist_ok=True)
poi_sufcapch_result.to_csv(file_path) 